# Experimental Hydrostatic Force Calculation

In [ ]:
import pandas as pd

# Find hydrostatic force in a specified time range (before force starts to decrease as water fills in the structure)
time_start = xx  # in seconds
time_end = xx     # in seconds
# Load the CSV file to read its content
file_path = # 'path_to_your_file.csv'
data = pd.read_csv(file_path)

import matplotlib.pyplot as plt
import numpy as np

# Experimental sampling rate 1000 Hz
sampling_rate = 1000  # Hz

# Create a time array based on the sampling rate and the number of samples
time = np.arange(len(data)) / sampling_rate

# Extract Force Z data
force_z = -data['Force Z (N)'] # negative because of the orientation of the load cell


# Plotting Force Z against Time
plt.figure(figsize=(12, 6))
plt.plot(time, force_z, label='Force Z (N)')
plt.xlabel('Time (s)')
plt.ylabel('Force Z (N)')
plt.title('Force Z over Time')
plt.legend()
# Plot vertical lines to indicate the specified time range
plt.axvline(time_start, color='r', linestyle='--', label='Start Time')
plt.axvline(time_end, color='g', linestyle='--', label='End Time')
plt.xlim(90,100)
plt.grid(alpha=0.15)
plt.show()

# Function to calculate the average Force Z in a specified time range
def average_force_z(time_start, time_end, force_z, sampling_rate):
    """
    Calculate the average Force Z between the specified time range.
    
    Parameters:
    time_start (float): Start time in seconds.
    time_end (float): End time in seconds.
    force_z (pd.Series): The Force Z data.
    sampling_rate (int): The sampling rate in Hz.
    
    Returns:
    float: The average Force Z in the specified time range.
    """
    # Convert time range to sample indices
    start_index = int(time_start * sampling_rate)
    end_index = int(time_end * sampling_rate)
    
    # Ensure indices are within the bounds of the data
    start_index = max(0, start_index)
    end_index = min(len(force_z), end_index)
    
    # Calculate and return the average Force Z
    return force_z[start_index:end_index].mean()



average_fz = average_force_z(time_start, time_end, force_z, sampling_rate)
average_fz


# T = period in seconds

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft, fftfreq
from scipy.signal import butter, filtfilt

# Assign hydrostatic force to a variable
hydrostatic_force = 0 

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y

# Function to read and process SPH data
def load_sph_data(filepath):
    # Reading the file which is semicolon-separated
    sph_data = pd.read_csv(filepath, delimiter=';')
    # Converting string numbers to float and handling scientific notation
    for col in ['ForceFluid.x [N]', 'ForceFluid.y [N]', 'ForceFluid.z [N]', 'ForceFluid [N]']:
        sph_data[col] = sph_data[col].astype(str).apply(lambda x: float(x.strip().replace('E', 'e')))
    
    # Calculate the average ForceFluid within a time range
    start_time, end_time = # both in seconds
    # Filter the data within the time range
    filtered_data = sph_data[(sph_data['Time [s]'] >= start_time) & (sph_data['Time [s]'] <= end_time)]
    # Calculate the average of the total force
    global hydrostatic_force 
    hydrostatic_force = filtered_data['ForceFluid [N]'].mean()
    # print(f'Average hydrostatic force between {start_time} and {end_time} seconds: {hydrostatic_force:.2f} N')
    # Returning processed data
    return sph_data

# Function to read and process Experimental data
def load_exp_data(filepath):
    # Reading the file which is comma-separated
    exp_data = pd.read_csv(filepath)
    # Add hydrostatic force to vertical force
    exp_data['Vertical Force (N)'] = -exp_data['Force Z (N)'] + average_fz # Get from the hydrostatic force experiment.
    # Use Force X and Force Y for horizontal forces
    exp_data['Horizontal Force X (N)'] = exp_data['Force X (N)']
    exp_data['Horizontal Force Y (N)'] = exp_data['Force Y (N)']
    # Calculate total force using Pythagorean theorem
    exp_data['Total Force (N)'] = np.sqrt(
        exp_data['Vertical Force (N)']**2 +
        exp_data['Horizontal Force X (N)']**2 +
        exp_data['Horizontal Force Y (N)']**2
    )
    # Print maximum total force value
    print(f'Maximum total force: {exp_data["Total Force (N)"].max():.2f} N')
   
 
    return exp_data

# Function to apply FFT filtering
def apply_fft_filter(data, time, cutoff_freq):
    # Calculate the sampling frequency
    fs = 1 / (time[1] - time[0])
    
    # Apply the lowpass filter
    filtered_data = butter_lowpass_filter(data, cutoff_freq, fs)
    
    return filtered_data

# Load and process data
sph_data = # load SPH data
exp_data = # load experimental data

# Create time steps for experimental data with 1000 Hz frequency
exp_time_steps = np.arange(0, len(exp_data) * 0.001, 0.001)

# Print hydrostatic force
print(f'Hydrostatic force: {hydrostatic_force:.2f} N')

# Apply butterworth filtering with specified cutoff frequencies. 
# Notice that for our case the filter did not change the signal much. 
sph_cutoff = xx # Hz, typically a high value such that not to filter anything as filter is not needed. 
exp_cutoff = xx # Hz, natural frequency of the structure.

sph_filtered = apply_fft_filter(sph_data['ForceFluid [N]'].values, sph_data['Time [s]'].values, sph_cutoff)
exp_filtered = apply_fft_filter(exp_data['Total Force (N)'].values, exp_time_steps, exp_cutoff)

# Plotting
plt.figure(figsize=(15,5))
plt.rcParams.update({'font.size': 20})
plt.plot(sph_data['Time [s]'] + xxx, sph_filtered*FF, label='SPH, $d_p=0.003m$', linestyle='-', color='green') # FF is the length scaling factor to match the experimental data, xxx is the time shift to match the experimental data
plt.plot(exp_time_steps, exp_filtered , label='Experiment', linestyle='--', color='black')
plt.xlabel('Time (s)', fontsize=25)
plt.xlim(19, 21.5)
plt.ylim(0,100)  # Adjusted y-axis limits for total force
plt.ylabel('Total Force on FSBW (N)', fontsize=25)
plt.legend(fontsize=20)
plt.grid(True, alpha=0.15)
plt.show()